# Knowledge Based Exercise

### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

# Read in the datasets
movies = pd.read_csv('data/movies_out.csv',index_col=0)
reviews = pd.read_csv('data/reviews_out.csv',index_col=0)

#### Part I: How To Find The Most Popular Movies?

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

Before you implement your code for `popular_recommendations` function, we will provide a helper function, called `create_ranked_df`. This helper function transforms `movies` and `reviews` dataframes into a `ranked_movies` dataframe of movies that are sorted by the highest average rating & time and have more than 4 ratings.

In [102]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,114508,8,1381006850,2013-10-05
1,2,208092,5,1586466072,2020-04-09
2,2,358273,9,1579057827,2020-01-15
3,2,10039344,5,1578603053,2020-01-09
4,2,6751668,9,1578955697,2020-01-13


In [103]:
movies.head()

,movie_id,movie,genre,year,1800s,1900s,2000s,Documentary,Short,Horror,...,Thriller,Animation,Music,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show
0,8,Edison Kinetoscopic Record of a Sneeze (1894),"['Documentary', 'Short']",1894,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,10,La sortie des usines Lumière (1895),"['Documentary', 'Short']",1895,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,12,The Arrival of a Train (1896),"['Documentary', 'Short']",1896,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),"['Short', 'Horror']",1896,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [104]:
reviews_agg = reviews.groupby('movie_id').agg({'rating':['mean','count'],
                                               'date':'max',
                                              })

col_names = []
for cols in reviews_agg.columns:
    col_names.append('_'.join(cols))

# rename columns
reviews_agg.columns = col_names

In [105]:
# join between movies and reviews
ranked_movies = movies.merge(reviews_agg, left_on='movie_id', right_index=True)

In [106]:
# just the movies with at least 5 ratings
ranked_movies = ranked_movies.loc[ranked_movies['rating_count']>=5]

In [107]:
ranked_movies.head()

,movie_id,movie,genre,year,1800s,1900s,2000s,Documentary,Short,Horror,...,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show,rating_mean,rating_count,date_max
74,12349,The Kid (1921),"['Comedy', 'Drama', 'Family']",1921,0,1,0,0,0,0,...,0,0,0,0,0,0,0,9.166667,6,2019-01-13
87,13442,Nosferatu (1922),"['Fantasy', 'Horror']",1922,0,1,0,0,0,1,...,0,0,0,0,0,0,0,9.000000,6,2018-09-30
97,14429,Safety Last! (1923),"['Comedy', 'Thriller']",1923,0,1,0,0,0,0,...,0,0,0,0,0,0,0,8.600000,5,2017-11-04
123,15864,The Gold Rush (1925),"['Adventure', 'Comedy', 'Drama', 'Family']",1925,0,1,0,0,0,0,...,0,0,0,0,0,0,0,8.875000,8,2019-01-20
139,17136,Metropolis (1927),"['Drama', 'Sci-Fi']",1927,0,1,0,0,0,0,...,0,0,0,0,0,0,0,9.000000,6,2018-09-28


In [108]:
ranked_movies = ranked_movies.sort_values(by=['rating_mean','rating_count','date_max'], ascending=[False,False,False])

In [109]:
ranked_movies.head()

,movie_id,movie,genre,year,1800s,1900s,2000s,Documentary,Short,Horror,...,Musical,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show,rating_mean,rating_count,date_max
31390,5512872,Be Somebody (2016),"['Comedy', 'Drama', 'Romance']",2016,0,0,1,0,0,0,...,0,0,0,0,0,0,0,10.000000,41,2018-05-17
2983,59113,Doctor Zhivago (1965),"['Drama', 'Romance', 'War']",1965,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.000000,5,2017-11-19
16239,986264,Taare Zameen Par (2007),"['Drama', 'Family']",2007,0,0,1,0,0,0,...,0,0,0,0,0,0,0,9.666667,18,2020-03-23
34862,9095324,Coldplay: A Head Full of Dreams (2018),"['Documentary', 'Music']",2018,0,0,1,1,0,0,...,0,0,0,0,0,0,0,9.555556,9,2020-02-08
275,21749,City Lights (1931),"['Comedy', 'Drama', 'Romance']",1931,0,1,0,0,0,0,...,0,0,0,0,0,0,0,9.500000,10,2019-05-31


In [110]:
def create_ranked_df(movies, reviews):
    reviews_agg = reviews.groupby('movie_id').agg({'rating':['mean','count'],
                                               'date':'max',
                                              })

    col_names = []
    for cols in reviews_agg.columns:
        col_names.append('_'.join(cols))

    # rename columns
    reviews_agg.columns = col_names

    # join between movies and reviews
    ranked_movies = movies.merge(reviews_agg, left_on='movie_id', right_index=True)

    # just the movies with at least 5 ratings
    ranked_movies = ranked_movies.loc[ranked_movies['rating_count']>=5]

    ranked_movies = ranked_movies.sort_values(by=['rating_mean','rating_count','date_max'], ascending=[False,False,False])

    ranked_movies['rank'] = np.arange(1,ranked_movies.shape[0]+1)

    return ranked_movies

In [111]:
np.arange(1,ranked_movies.shape[0]+1)

array([   1,    2,    3, ..., 3432, 3433, 3434])

In [112]:
create_ranked_df(movies, reviews)

,movie_id,movie,genre,year,1800s,1900s,2000s,Documentary,Short,Horror,...,Film-Noir,Adult,Talk-Show,News,Reality-TV,Game-Show,rating_mean,rating_count,date_max,rank
31390,5512872,Be Somebody (2016),"['Comedy', 'Drama', 'Romance']",2016,0,0,1,0,0,0,...,0,0,0,0,0,0,10.000000,41,2018-05-17,1
2983,59113,Doctor Zhivago (1965),"['Drama', 'Romance', 'War']",1965,0,1,0,0,0,0,...,0,0,0,0,0,0,10.000000,5,2017-11-19,2
16239,986264,Taare Zameen Par (2007),"['Drama', 'Family']",2007,0,0,1,0,0,0,...,0,0,0,0,0,0,9.666667,18,2020-03-23,3
34862,9095324,Coldplay: A Head Full of Dreams (2018),"['Documentary', 'Music']",2018,0,0,1,1,0,0,...,0,0,0,0,0,0,9.555556,9,2020-02-08,4
275,21749,City Lights (1931),"['Comedy', 'Drama', 'Romance']",1931,0,1,0,0,0,0,...,0,0,0,0,0,0,9.500000,10,2019-05-31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26125,3079016,We'll Never Have Paris (2014),"['Comedy', 'Romance']",2014,0,0,1,0,0,0,...,0,0,0,0,0,0,3.200000,5,2015-06-17,3430
9263,118688,Batman & Robin (1997),"['Action', 'Sci-Fi']",1997,0,1,0,0,0,0,...,0,0,0,0,0,0,3.000000,8,2017-12-10,3431
16703,1085492,The Prince (2014),"['Action', 'Thriller']",2014,0,0,1,0,0,0,...,0,0,0,0,0,0,3.000000,5,2016-11-06,3432
31687,5690360,Slender Man (2018),"['Horror', 'Mystery', 'Thriller']",2018,0,0,1,0,0,1,...,0,0,0,0,0,0,1.571429,7,2019-05-06,3433


In [113]:
def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a dataframe from 

    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    top_movies = ranked_movies.iloc[:n_top]
    
    return top_movies

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace. 

In [114]:
# Top 20 movies recommended for id 1
recs_20_for_1 = popular_recommendations(1, 20, ranked_movies)
recs_20_for_1 = recs_20_for_1['movie'].to_list()

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968, 5, ranked_movies)
recs_5_for_53968 = recs_5_for_53968['movie'].to_list()

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000, 100, ranked_movies)
recs_100_for_70000 = recs_100_for_70000['movie'].to_list()

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43, 35, ranked_movies)
recs_35_for_43 = recs_35_for_43['movie'].to_list()

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace. 

In [115]:
### You Should Not Need To Modify Anything In This Cell

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!

### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

In [116]:
def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
    '''
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    
    # Implement your code here
    if genres != None:
        ranked_movies = ranked_movies.loc[ranked_movies[genres].sum(axis=1)>0]
    if years != None:    
        ranked_movies = ranked_movies.loc[np.isin(ranked_movies['year'].astype('str'), years)]

    # Step 1: filter movies based on year and genre 
    # Step 2: create top movies list 
    top_movies = ranked_movies.iloc[:n_top]
    
    return top_movies['movie'].to_list()

In [117]:
# Top 20 movies recommended for id 1 with years=['2015', '2016', '2017', '2018'], genres=['History']
years=['2015', '2016', '2017', '2018']
genres=['History']
recs_20_for_1_filtered = popular_recs_filtered(1, 20, ranked_movies, years=years, genres=genres)

# Top 5 movies recommended for id 53968 with no genre filter but years=['2015', '2016', '2017', '2018']
years=['2015', '2016', '2017', '2018']
recs_5_for_53968_filtered = popular_recs_filtered(53968, 5, ranked_movies, years=years, genres=None)

# Top 100 movies recommended for id 70000 with no year filter but genres=['History', 'News']
genres=['History', 'News']
recs_100_for_70000_filtered = popular_recs_filtered(70000, 100, ranked_movies, years=None, genres=genres)

In [122]:
### You Should Not Need To Modify Anything In This Cell

# check 1 
assert t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History']) == recs_20_for_1_filtered,  "The first check failed..."
# check 2
assert t.popular_recs_filtered('53968', 5, ranked_movies, years=['2015', '2016', '2017', '2018']) == recs_5_for_53968_filtered,  "The second check failed..."
# check 3
assert t.popular_recs_filtered('70000', 100, ranked_movies, genres=['History', 'News']) == recs_100_for_70000_filtered,  "The third check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

KeyError: 'date'